In [1]:
#Find a way to save the tweet to disk

import json
import oauth2 as oauth
import os
import emoji as emo
import urllib.parse as parse
import csv
from sklearn.feature_extraction.text import CountVectorizer
from stemming.porter2 import stem
from string import punctuation
import numpy
import pymongo
import re
import unicodedata
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

In [2]:
#This function will use an online ready-made dictionary to find the sentiment of an emoji

#load csv info into dictionary
emoji_scores = {}
with open('Emoji_Sentiment_Data_v1.0.csv', newline = '') as emoji_csv:
    emoji_reader = csv.reader(emoji_csv)
    next(emoji_reader, None)
    for row in emoji_reader:
        emoji = row[0]
        neg_score = float(int(row[4])/int(row[2]))
        neut_score = float(int(row[5])/int(row[2]))
        pos_score = float(int(row[6])/int(row[2]))
        sent_score = (-1 * neg_score + pos_score) - .305
        emoji_scores[emoji] = sent_score
        
#print(emoji_scores)
        

In [3]:
consumer_key = os.environ.get('CONSUMER_KEY')
consumer_secret = os.environ.get('CONSUMER_SECRET')

access_token = os.environ.get('ACCESS_TOKEN')
access_token_secret = os.environ.get('ACCESS_TOKEN_SECRET')

consumer = oauth.Consumer(key=consumer_key, secret=consumer_secret)
access_token = oauth.Token(key=access_token, secret=access_token_secret)
client = oauth.Client(consumer, access_token)



emo_tweets = {}
for i in range(0, 10):
    params = {'q' : 'Justin Bieber', 'count' : 100, 'lang' : 'en'}
    # See Twitter Search API documenation for description of cursoring for below if block
    if i > 0:
        params['max_id'] = max_id 
    timeline_endpoint_baseuri = 'https://api.twitter.com/1.1/search/tweets.json?'
    timeline_endpoint = timeline_endpoint_baseuri + parse.urlencode(params)
    print(timeline_endpoint)
    response, data = client.request(timeline_endpoint)
    tweets_results = json.loads(data)
    tweets_list = tweets_results['statuses']
    max_id = tweets_list[len(tweets_list) - 1]['id'] - 1
    for tweet in tweets_list:
        tweet_emojis = []
        for char in tweet['text']:
            if char in emo.UNICODE_EMOJI:
                tweet_emojis.append(char)
        if tweet_emojis:
            for emoji in tweet_emojis:
                tweet_emoji_scores = []
                try:
                    tweet_emoji_scores.append(emoji_scores[emoji])
                except KeyError as e:
                    print('No emoji %s' % e)
                    continue
            if tweet_emoji_scores:
                if max(tweet_emoji_scores) == max(numpy.absolute(tweet_emoji_scores)):
                    tweet_score = max(tweet_emoji_scores)
                else:
                    tweet_score = min(tweet_emoji_scores)
                emo_tweets[tweet['id']] = [tweet['text'], tweet_score]
    print('Emoji tweets processed: ' + str(len(emo_tweets.keys())))
            

for tweet in emo_tweets.values():
    print(tweet)
    print()

ValueError: Key and secret must be set.

In [25]:
# text preprocessing (partially from before partially from August)
mongo_uri = 'mongodb://heroku_xgnhblcr:' + os.environ.get('MONGODB_PASSWORD') + '@ds149511.mlab.com:49511/heroku_xgnhblcr'
client = pymongo.MongoClient(mongo_uri)
db = client.get_default_database()
myresults = list(db.test.find())
tweets = []
emoji_tweets = []
for entry in myresults:
    tweets.append(entry['text'])
#Doing the next part based on the earlier REST-API-using cell
for tweet in tweets:
    tweet_emojis = []
    for char in tweet:
        if char in emo.UNICODE_EMOJI:
            tweet_emojis.append(char)
    if tweet_emojis:
        for emoji in tweet_emojis:
            tweet_emoji_scores = []
            try:
                tweet_emoji_scores.append(emoji_scores[emoji])
            except KeyError as e:
                print('No emoji score for %s' % e)
                continue
            if tweet_emoji_scores:
                if max(tweet_emoji_scores) == max(numpy.absolute(tweet_emoji_scores)):
                    tweet_score = max(tweet_emoji_scores)
                else:
                    tweet_score = min(tweet_emoji_scores)
                emoji_tweets.append([tweet, tweet_score])
    
print(emoji_tweets)

No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏽'
No emoji score for '🤷'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🤦'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🙂'
No emoji score for '🏾'
No emoji score for '🤷'
No emoji score for '🏻'
No emoji score for '🏾'
No emoji score for '🤷'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '‼'
No emoji score for '🏾'
No emoji score for '🤦'
No emoji score for '🤦'
No emoji score for '🤘'
No emoji score for '🙂'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🤷'
No emoji score for '🏾'
No emoji score for '🏾'
No emoji score for '🤷'
No emoji sc

In [26]:
tweet_text= []
for tweet in emoji_tweets:
    tweet_text.append(tweet[0])
    
    
def preprocessor(tweet):
    
    #remove accents
    normalized = unicodedata.normalize('NFKD', tweet)
    if normalized != tweet:
        tweet = ''.join([c for c in normalized if not unicodedata.combining(c)])
        
    tweet = tweet.lower()
    
    #remove URLs
    tweet = re.sub(r'https?:\/\/.*?(\s|$)', '', tweet)
    tweet = re.sub(r'@\w*?(\s|$)', '', tweet)
    tweet = re.sub(r'#\w*?(\s|$)', '', tweet)
    
    #remove RT (retweet) - this doesn't seem to work since 'rt' is in the preprocessed data
    tweet = re.sub('^rt\s', '', tweet)
    
    #Note - the preprocessor already seems to be removing emojis - no need for extra code
    
    # Regularize apostrophes from different parts of unicode
    tweet = re.sub('|'.join(['‘', '’', 'ʻ', 'ʼ']), '\'', tweet)
    
    return tweet
    


In [27]:
def tokenizer(tweet): #Currently troubleshooting function
    
    tokens = []
    token_pattern = '(^|(?!\w).)((\w|\')+)\W'
    token_matches = re.finditer(token_pattern, tweet)
    for token_match in token_matches:
        tokens.append(token_match.group(2))
    return tokens

print(tokenizer("Hard pass...I'll give as much respect as y'all did Barack...none 😒 https://t.co/UJcr4mrZRA"))

['Hard', "I'll", 'as', 'respect', "y'all", 'Barack', 'none', 'https', 't']


In [28]:
vectorizer = CountVectorizer(preprocessor=preprocessor, tokenizer=tokenizer) #Make binary?  Does this prevent it from picking up one word twice in a tweet?
fit_tr = vectorizer.fit_transform(tweet_text)
print(vectorizer.get_feature_names())
print(fit_tr)
#   for word in emoji_tweets.split(): #This and following line should be removed and use regex instead.  Follow Downloads/pak-paroubek.pdf for preprocessing ideas, as well as downloads that were downloaded at a similar time
#      if word[0:4] == 'http':
            
#CountVectorizer().build_tokenizer()(tweet_text) #stop_words param to 'english' once you have only Eng tweets, this was indented within for loop at the beginning of August

['1', '1st', '200', '2498498', '2k18', '500', '54ange', '_itswish', 'a', 'able', 'across', 'activate', 'actually', 'after', "ain't", 'all', 'alone', 'already', 'also', 'although', 'always', 'amazing', 'amp', 'an', 'and', 'angrezi', 'anything', 'anyways', 'are', 'around', 'as', 'away', 'back', 'bae', 'bayizzat', 'baysharm', 'be', 'beef', 'being', 'best', 'bigsexy_tote', 'blade', 'blake', 'both', 'bout', 'business', 'but', 'by', 'can', "can't", 'care', 'cats', 'cause', 'characters', 'chirs', 'chupacaubrey', 'classes', 'code', 'comments', 'concept', 'congrats', 'contract', 'cool', 'coz', 'crackhead', 'ctrl', 'cuffed', 'daily', 'dammit', 'dat', 'deserved', 'disaster', 'djyunggvirgil', 'do', "don't", 'dont', 'drinking', 'either', 'elfonthebeat', 'elliegmeme', 'enter', 'even', 'ever', 'explain', 'extra', 'f', 'faris', 'favstar_bot', 'feeling', 'felt', 'find', 'first', 'fly', 'for', 'foziapti', 'frandship', 'from', 'fuckers', 'get', 'gettin', 'getting', 'girls', 'give', 'go', 'god', 'going', 

In [29]:
transformer = TfidfTransformer()  # This block is not correct
tfidf = transformer.fit_transform(fit_tr)

In [30]:
    tweet_emos = []
    for tweet in emoji_tweets:
        tweet_emos.append(tweet[1] > 0)
        
    
    #Cross-validation
    from sklearn.model_selection import cross_val_score
    print(type(tfidf.toarray()))
          
    scores = cross_val_score(MultinomialNB(alpha=.01), tfidf, tweet_emos, cv=5)
    print(scores)  
    
    

<class 'numpy.ndarray'>
[ 0.52830189  0.66666667  0.78431373  0.82352941  0.62745098]


In [ ]:
    
    
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    MultinomialNB(alpha=.01).fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)